## 1. Configuração e Carregamento

In [12]:
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
# Vamos recomendar o Top 3 (K=3)
K = 3 

#O que é mainstreaming? as provincias mais populares
MAINSTREAMING = 2
# O que é "Sustentável"? O impacto ambiental acima de 0.7
SUSTENTAVEL = 0.7

print("Bibliotecas carregadas e configurações definadas.")

Bibliotecas carregadas e configurações definadas.


## 2. Carregar Dados

In [13]:
# Carregar os nossos ficheiros de "combustível"
try:
    df_provincias = pd.read_csv('../data/provincia_features_MVP.csv', index_col='Provincia')
    df_interactions = pd.read_csv('../data/user_interactions_MVP.csv')
    print("Ficheiros MVP carregados com sucesso.")
except FileNotFoundError:
    print("ERRO: Ficheiros MVP não encontrados na pasta /data/!")

print("\n--- Features das Províncias (Amostra) ---")
print(df_provincias.head(3))

Ficheiros MVP carregados com sucesso.

--- Features das Províncias (Amostra) ---
           popularidade  custo_medio  impacto_ambiental  feat_cultura  \
Provincia                                                               
Luanda           137000          300                0.2           1.0   
Benguela          97000          220                0.4           0.5   
Huíla             73000          100                0.7           0.7   

           feat_praia  feat_aventura  feat_natureza  
Provincia                                            
Luanda            0.8            0.1            0.2  
Benguela          1.0            0.5            0.3  
Huíla             0.0            0.8            1.0  


## 3. Definir o Modelo Baseline (Popularidade)

In [14]:
def get_baseline_recommendations(df_prov, k):
    """
    Retorna as K províncias mais populares.
    """
    # Ordenar por 'popularidade' (do maior para o menor)
    recommendations = df_prov.sort_values(by='popularidade', ascending=False).index.tolist()
    # Retornar apenas o Top-K
    return recommendations[:k]

# Obter as recomendações (serão as mesmas para todos)
baseline_recs = get_baseline_recommendations(df_provincias, K)

print(f"O Modelo Baseline recomenda a TODOS os utilizadores: {baseline_recs}")

O Modelo Baseline recomenda a TODOS os utilizadores: ['Luanda', 'Benguela', 'Huíla']


## 4. Avaliação (Comparar com a Realidade)

In [15]:
# 1. Obter o "Ground Truth" (O que os utilizadores REALMENTE gostaram - ratings >= 4)
ground_truth = {}
for user in df_interactions['user_id'].unique():
    user_likes = df_interactions[
        (df_interactions['user_id'] == user) & (df_interactions['rating'] >= 4)
    ]['provincia_visitada'].tolist()
    ground_truth[user] = user_likes

print("\n--- 'Verdade' (O que os utilizadores gostaram) ---")
print(ground_truth)


# 2. Calcular Métricas de Relevância (Precisão, Recall)
lista_precision = []
lista_recall = []

for user, likes in ground_truth.items():
    if not likes:
        continue # Ignorar utilizadores sem 'likes'

    # Quantos 'hits' (acertos) o modelo teve?
    hits = len(set(baseline_recs) & set(likes))
    
    # Precisão = (Acertos) / (Total Recomendado)
    precision_k = hits / K
    lista_precision.append(precision_k)
    
    # Recall = (Acertos) / (Total que o utilizador Gostou)
    recall_k = hits / len(likes)
    lista_recall.append(recall_k)

print("\n--- Métricas de Relevância (Média) ---")
print(f"Precisão@ {K} (Média): {np.mean(lista_precision) * 100:.2f}%")
print(f"Recall@ {K} (Média):   {np.mean(lista_recall) * 100:.2f}%")


--- 'Verdade' (O que os utilizadores gostaram) ---
{np.int64(1): ['Luanda', 'Benguela'], np.int64(2): ['Malanje', 'Cuando Cubango'], np.int64(3): ['Malanje', 'Huambo']}

--- Métricas de Relevância (Média) ---
Precisão@ 3 (Média): 22.22%
Recall@ 3 (Média):   33.33%


## 5. Métricas de Impacto

In [16]:
# 1. Definir o que é "Emergente" vs "Sustentável"
mainstream_list = df_provincias.sort_values(by='popularidade', ascending=False).head(MAINSTREAMING).index.tolist()
sustainable_list = df_provincias[df_provincias['impacto_ambiental'] >= SUSTENTAVEL].index.tolist()

print(f"\nDefinido como 'Mainstream': {mainstream_list}")
print(f"Definido como 'Sustentável': {sustainable_list}")

# 2. Calcular as métricas
emergentes_recomendados = len(set(baseline_recs) - set(mainstream_list))
sustentaveis_recomendados = len(set(baseline_recs) & set(sustainable_list))

diversidade_percent = (emergentes_recomendados / K) * 100
sustentabilidade_percent = (sustentaveis_recomendados / K) * 100

print("\n--- MÉTRICAS DE NEGÓCIO ---")
print(f"Diversidade@ {K} (% Emergentes):   {diversidade_percent:.2f}%")
print(f"Sustentabilidade@ {K} (% Sustentáveis): {sustentabilidade_percent:.2f}%")


Definido como 'Mainstream': ['Luanda', 'Benguela']
Definido como 'Sustentável': ['Huíla', 'Namibe', 'Malanje', 'Cabinda', 'Cuando Cubango']

--- MÉTRICAS DE NEGÓCIO ---
Diversidade@ 3 (% Emergentes):   33.33%
Sustentabilidade@ 3 (% Sustentáveis): 33.33%


## 6. Conclusões: A Prova do Problema

Com `K=3`, o Modelo Baseline (Popularidade) recomenda `['Luanda', 'Benguela', 'Huíla']` a todos os utilizadores. A análise das métricas prova que este modelo falha em todos os objectivos da plataforma.

### 1. Falha na Relevância (Métricas de Performance)

* **`Precisão@3` (Média): 22.22%**
* **`Recall@3` (Média): 33.33%**

**Análise:** O modelo falha catastroficamente para utilizadores "Exploradores".
* Ele tem 0% de precisão para o **User 2** (que gostava de `['Malanje', 'Cuando Cubango']`).
* Ele tem 0% de precisão para o **User 3** (que gostava de `['Malanje', 'Huambo']`).
O modelo "acerta" (parcialmente) apenas no User 1 (Mainstream), provando que é um modelo "tamanho único" que ignora a personalização.

### 2. Falha nos Objectivos (Métricas de Plataforma e Impacto)

* **`Diversidade@3` (% Emergentes): 33.33%**
* **`Sustentabilidade@3` (% Sustentáveis): 33.33%**

**Análise:** O modelo recomenda 1 destino "emergente" (`Huíla`) e 1 "sustentável" (`Huíla`). Estes números, embora não sejam 0%, são um *acidente* de popularidade, não uma *intenção* do modelo. O sistema continua a reforçar os destinos "mainstream" (`Luanda`, `Benguela`) e falha em promover activamente a diversidade e a sustentabilidade.

**Próximo Passo:**
Provámos que a popularidade é um "inimigo" da personalização e do impacto. Vamos agora construir o `03_Content_Based_Model.ipynb` para provar que conseguimos vencer este Baseline em todas as métricas.